Run the following cell to install the required packages for running the qGAN

In [ ]:
!pip install qiskit==0.34.0 qiskit_machine_learning==0.3.0 qiskit_finance==0.3.0 qiskit-aer-gpu==0.10.2 --quiet
!pip3 install torch==1.8.1

In [1]:
#import sys
#sys.path.append('.')
import os
from scipy import stats
import numpy as np
import pandas as pd
import logging
import time
import pandas as pd
import qiskit
import matplotlib.pyplot as plt
from utils import experiment_utils
from utils.custom_discriminator import CustomPyTorchDiscriminator
from qiskit.circuit.library import TwoLocal
from qiskit.opflow.gradients import Gradient
from qiskit_finance.circuit.library import UniformDistribution
from qiskit_machine_learning.algorithms import QGAN

%matplotlib inline

In [2]:
snapshot_dir = 'data/qgan-replication/'
experiment_utils.check_create_dir(snapshot_dir)

# Setting up Gan logger
gan_logger = logging.getLogger('qiskit_machine_learning.algorithms.distribution_learners.qgan')
formatter = logging.Formatter('%(asctime)s - gan-logger - %(message)s')
s_handler = logging.StreamHandler()
file_handler = logging.FileHandler(os.path.join(snapshot_dir, 'experiment.log'), mode='+w')
gan_logger.addHandler(file_handler)
s_handler.setFormatter(formatter)
gan_logger.setLevel(logging.DEBUG)
gan_logger.addHandler(s_handler)

In [3]:
# Experiment settings
n_samples = 20000

simulator_device, torch_device = 'GPU', 'cuda:0'

# Gan configurations
batch_size = 2000
n_epochs = 1000
n_runs = 1
seed = 7
n_qubits = 7
value_bounds = [0., 20.]
rng = np.random.default_rng(seed)

# Backends
quantum_instance = experiment_utils.build_quantum_instance(
                    'statevector_simulator',
                    seed=seed,
                    n_shots=8192,
                    device=simulator_device)

In [9]:
def run_experiment(
  dataset, 
  x_pmf,
  value_bounds,
  n_qubits,
  batch_size,
  n_epochs,
  quantum_instance, 
  exp_dir,
  seed, 
  torch_device):

  # Setting up discriminator
  discriminator = CustomPyTorchDiscriminator(device=torch_device)

  # Setting up generator
  init_circ = UniformDistribution(n_qubits)
  gen_circ = TwoLocal(n_qubits, ['ry', 'rz'], 'cx', reps=1)
  gen_circ.compose(init_circ, front=True, inplace=True)

  # Setting up qGAN
  qgan = QGAN(
            dataset,
            value_bounds, 
            num_qubits=[n_qubits], 
            batch_size=batch_size, 
            num_epochs=n_epochs,
            discriminator=discriminator,
            quantum_instance=quantum_instance,
            snapshot_dir=exp_dir,
            seed=seed
          )

  qgan.set_generator(gen_circ)

  toc = time.time()
  results = qgan.run()
  tic = time.time()

  print('Duration (min): ', (tic-toc) / 60 )

  qgan.generator._gradient_function = None
  if exp_dir:
    experiment_utils.serialize_model(os.path.join(exp_dir,'qgan.pkl'), qgan)

  # Generating probabilities
  _, gen_probs = qgan.generator.get_output(quantum_instance)

  # Computing fidelity
  generated_state = experiment_utils.extract_statevector_from_generator(quantum_instance, qgan.generator)

  input_state = np.sqrt(x_pmf)
  input_state = input_state / np.linalg.norm(input_state)
  computed_fidelity = experiment_utils.fidelity(input_state, generated_state)

  # Plotting results
  experiment_utils.plot_probs(
    x_pmf, 
    gen_probs, 
    computed_fidelity, 
    x=np.linspace(0., value_bounds[1], 2**n_qubits),
    experiment_dir=exp_dir)
  
  # Saving frequencies
  dict_freqs = { 
                'discretization-pmf': x_pmf,
                'real-data-freqs':  qgan._prob_data,
                'generated-freqs': gen_probs 
               }
               
  file_name = os.path.join(exp_dir, 'frequencies.csv')
  pd.DataFrame(dict_freqs).to_csv(file_name, index=False)

  # Plotting GAN's performance
  rel_entr = qgan.rel_entr
  experiment_utils.plot_relative_entropy(rel_entr, experiment_dir=exp_dir)

  g_loss = np.array(qgan.g_loss).reshape(-1)
  d_loss = np.array(qgan.d_loss).reshape(-1)
  experiment_utils.plot_gan_losses(g_loss, d_loss, experiment_dir=exp_dir)

  # Saving performances
  dict_metrics_per_epoch = {
    'g-loss': g_loss, 
    'd-loss': d_loss, 
    'rel-entr': rel_entr
  }

  file_name = os.path.join(exp_dir, 'losses_rel_entr.csv')
  pd.DataFrame(dict_metrics_per_epoch).to_csv(file_name, index=False)

# Lognormal distribution
Varying the scale parameter of the distribution in $\{0.5, 1, 2\}$

In [ ]:
scales_list = [0.5, 1, 2]

gan_logger.info('Experiment for the LOGNORMAL distribution')

for scale in scales_list:

  gan_logger.info(f'Scale {scale}')

  exp_dir = os.path.join(snapshot_dir, f'lognormal_s{str(scale)}')
  experiment_utils.check_create_dir(exp_dir)

  # Sampling dataset
  loc = 0
  dataset = experiment_utils.sample_from_distribution(
              'lognormal', 
              n_samples,
              loc, 
              scale,
              random_state=rng
            )

  dataset %= value_bounds[1]

  # Estimating PMF
  x = np.linspace(0., value_bounds[1], 2**n_qubits)
  distance = (x.max() - x.min()) / (2**n_qubits - 1)
  delta = distance * .5
  xcdf_plus = stats.lognorm.cdf(x + delta, s=1, scale=scale)
  xcdf_minus = stats.lognorm.cdf(x - delta, s=1, scale=scale)
  x_pmf = xcdf_plus - xcdf_minus

  run_experiment(
    dataset,
    x_pmf,
    value_bounds,
    n_qubits,
    batch_size,
    n_epochs,
    quantum_instance,
    exp_dir,
    seed,
    torch_device
  )

# Laplace, Normal and Semicircular distributions

Running one experiment for each distribution with specific parameters

In [ ]:
gan_logger.info('Experiment for the LOGNORMAL distribution')

distributions_params = { 
  'laplace': { 'loc':} 
}

for scale in scales_list:

  gan_logger.info(f'Scale {scale}')

  exp_dir = os.path.join(snapshot_dir, f'lognormal_s{str(scale)}')
  experiment_utils.check_create_dir(exp_dir)

  # Sampling dataset
  loc = 0 
  #scale = 0.5
  dataset = experiment_utils.sample_from_distribution(
              'lognormal', 
              n_samples,
              loc, 
              scale,
              random_state=rng
            )

  dataset %= value_bounds[1]

  # Estimating PMF
  x = np.linspace(0., value_bounds[1], 2**n_qubits)
  distance = (x.max() - x.min()) / (2**n_qubits - 1)
  delta = distance * .5
  xcdf_plus = stats.lognorm.cdf(x + delta, s=1, scale=scale)
  xcdf_minus = stats.lognorm.cdf(x - delta, s=1, scale=scale)
  x_pmf = xcdf_plus - xcdf_minus

  run_experiment(
    dataset,
    x_pmf,
    value_bounds,
    n_qubits,
    batch_size,
    n_epochs,
    quantum_instance,
    exp_dir,
    seed,
    torch_device
  )